# Data Science Notebook

Build, train and serialize the model

# Import packages

In [48]:
# load data
from submodules.load_data import load_data
# data manipulation
import numpy as np
import pandas as pd

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from submodules.plots import plotGender
from submodules.plots import plotUnit


# data splitting
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

# data preprocessing
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

# model
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate


from xgboost import XGBClassifier


# performance
from sklearn.metrics import f1_score

# Load the data

Load semi-colon seperated data from disk

In [2]:
data = load_data()

# Create a Test Dataset
> uses scikit-learn

Performing this early minimizes generalization and bias you may inadvertently apply to your system.
Simply put, a test set of data involves: picking ~20% of the instances randomly and setting them aside.

Some considerations for sampling methods that generate the test set:
1. you don't want your model to see the entire dataset
1. you want to be able to fetch new data for training
1. you want to maintain the same percentage of training data against the entire dataset
1. you want a representative training dataset (~7% septic positive)

https://realpython.com/train-test-split-python-data/

In [3]:
# sets 20% of the data aside for testing, sets the random number generate to it always generates the same shuffled indicies
# x = 2 dimensional array with inputs
# X_train is the training part of the first sequence (x)
# X_test is the test part of the first sequence (x)
# y = 1 dimensional array with outputs
# y_train is the labeled training part of the second sequence
# y_test is the labeled test part of the second sequence
# test_size is the amount of the total dataset to set aside for testing
# random state fixes the randomization so you get the same results each time
# Shuffle before the data is split, it is shuffled
# stratified splitting keeps the proportion of y values trhough the train and test sets
X_train, X_test, y_train, y_test = \
    train_test_split(data.drop("isSepsis", axis=1),
    data["isSepsis"], test_size=0.2,
    random_state=42, stratify=data["isSepsis"])

# Feature Engineering

Age
- divide into age groups for both training and test datasets
- under 19 years old
- 19 to 65 years old
- 65 years and older

In [4]:
# X_train is all the instance with attributes
X_train = X_train.loc[X_train["Age"] <= 100]
# y_train is the label of each instance (isSepsis = 1 or 0)
y_train = y_train.loc[X_train["Age"] <= 100]

# X_test is all the instance with attributes
X_test = X_test.loc[X_test["Age"] <= 100]
# y_test is the label of each instance (isSepsis = 1 or 0)
y_test = y_test.loc[X_test["Age"] <= 100]

# Data Cleaning & Transformation Pipeline

## Age
- divide age into categories

In [5]:
# see the shape of the data before transformation
print("Shape of Age data before transforming: ", X_train["Age"].shape)

Shape of Age data before transforming:  (29041,)


In [6]:
# discretization is the process of transferring continuous
# functions, models, variables, and equations into discrete
# counterparts. This process is usually carried out as a first
# step toward making them suitable for numerical evaluation
# and implementation on digital computers.
# this splits age into 4 categories
def discretizateAge(data):
    # teen, youth, adult, senior
    bins = [13, 18, 30, 60, np.inf]
    data = np.digitize(data, bins=bins)
    data = data.reshape(len(data), 1)
    return data

# from scikit-learn
# transform() method transforms the entire dataset
# which is passes as a parameter to the function
DiscretizateAge = FunctionTransformer(discretizateAge)
# fits the transformation back to the dataset
DiscretizateAge.fit_transform(X_train["Age"]).shape

(29041, 1)

In [7]:
# scikit-learn provides a pipeline class
# create a class to replace missing values with the median of the column data
# fit the imputer instance to the  data using fit() method
age_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("discretizator", DiscretizateAge)
])

# see the shape of the data
# use the "trained" imputer to transform the training set by replacing missing values with the learned medians
age_pipeline.fit_transform(X_train[["Age"]]).shape

(29041, 1)

## Intensive Care Units

Unit 1 and Unit 2
Unit1 - Administrative identifier for ICU unit (MICU)
Unit2 - Administrative identifier for ICU unit (SICU)

In [8]:
def CombineUnits(units_cols):
    data = units_cols.copy()
    data["Unit"] = pd.Series(np.zeros((len(data))))
    data.loc[data["Unit1"] == 1, "Unit"] = "MICU"
    data.loc[data["Unit2"] == 1, "Unit"] = "SICU"
    data.loc[(data["Unit1"].isna()) & (data["Unit2"].isna()), "Unit"] = "Other ICU"
    return data[["Unit"]]

In [9]:
combineUnit1and2 = FunctionTransformer(CombineUnits)

units = ["Unit1", "Unit2"]

unit_pipeline = Pipeline([
    ("combine", combineUnit1and2),
    ("encoder", OneHotEncoder(sparse=False))
])

unit_pipeline.fit_transform(X_train[units]).shape

(29041, 3)

In [10]:
acidbase_features = ["BaseExcess", "PaCO2"]

def isAcidBaseDisturb(cols):
    cols = np.c_[cols, np.zeros(len(cols))]
    cols[:,2][(cols[:,0] < -2) & (cols[:,1] < 40)] = 1
    col = cols[:,2].reshape(len(cols), 1)
    return col

FindAcidosis = FunctionTransformer(isAcidBaseDisturb)
FindAcidosis.fit_transform(X_train[acidbase_features])

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [11]:
acidbase_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("acidosis", FindAcidosis)
])

acidbase_pipeline.fit_transform(X_train[acidbase_features]).shape

(29041, 1)

In [12]:
num_features = ["HR",
                "O2Sat",
                "Temp",
                "MAP",
                "Resp",
                "AST",
                "BUN",
                "Alkalinephos",
                "Calcium",
                "Creatinine",
                "Glucose",
                "Bilirubin_total",
                "Hgb",
                "PTT",
                "WBC",
                "Fibrinogen",
                "Platelets",
                "ICULOS"
                ]

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

num_pipeline.fit_transform(X_train[num_features]).shape

(29041, 18)

In [13]:
gender_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder())
])

gender_pipeline.fit_transform(X_train[["Gender"]])

array([[1.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [0.]])

In [14]:
preprocessing_pipeline = ColumnTransformer([
    ("numbers", num_pipeline, num_features),
    ("acidbase", acidbase_pipeline, acidbase_features),
    ("age", age_pipeline, ["Age"]),
    ("units", unit_pipeline, units),
    ("gender", gender_pipeline, ["Gender"])
], verbose=True)

preprocessing_pipeline.fit_transform(X_train).shape

[ColumnTransformer] ....... (1 of 5) Processing numbers, total=   0.1s
[ColumnTransformer] ...... (2 of 5) Processing acidbase, total=   0.0s
[ColumnTransformer] ........... (3 of 5) Processing age, total=   0.0s
[ColumnTransformer] ......... (4 of 5) Processing units, total=   0.0s
[ColumnTransformer] ........ (5 of 5) Processing gender, total=   0.0s


(29041, 24)

# Data Pipeline against Train and Test Datasets

In [15]:
X_train = preprocessing_pipeline.fit_transform(X_train)
X_test = preprocessing_pipeline.fit_transform(X_test)

[ColumnTransformer] ....... (1 of 5) Processing numbers, total=   0.1s
[ColumnTransformer] ...... (2 of 5) Processing acidbase, total=   0.0s
[ColumnTransformer] ........... (3 of 5) Processing age, total=   0.0s
[ColumnTransformer] ......... (4 of 5) Processing units, total=   0.0s
[ColumnTransformer] ........ (5 of 5) Processing gender, total=   0.0s
[ColumnTransformer] ....... (1 of 5) Processing numbers, total=   0.0s
[ColumnTransformer] ...... (2 of 5) Processing acidbase, total=   0.0s
[ColumnTransformer] ........... (3 of 5) Processing age, total=   0.0s
[ColumnTransformer] ......... (4 of 5) Processing units, total=   0.0s
[ColumnTransformer] ........ (5 of 5) Processing gender, total=   0.0s


# Model Selection

![image](images/scikirlearn-choose-right-estimator.png)

https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

1. [Linear Support Vector Machine](https://scikit-learn.org/stable/modules/svm.html#classification)
1. [Logistic Regression](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression) a linear model for classification
1. [K-Neighbors Classifier](https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification) implements learning based on the  nearest neighbors of each query point, where  is an integer value specified by the user.
1. [Random Forest Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.
1. XG Boost Classifier
1. [Neural Network Multi-Layer Perceptron Classifier](https://scikit-learn.org/stable/modules/neural_networks_supervised.html#multi-layer-perceptron) a supervised learning algorithm that learns a function  by training on a dataset

## Scoring
Validation output readings"
    - fit_time - the time for fitting the estimator on the train set for each cv split
    - score_time - the time for scoring the estimator on the test set for each cv split
    - test_score - The score array for test scores on each cv split
    - train_score - The score array for train scores on each cv split.

## Linear SVM

In [46]:
# select the SVM
lin_svm = svm.SVC()
# fit the model to the data
lin_svm.fit(X_train, y_train)
# configure the cross valudation
cv_lin_svm = cross_validate(lin_svm, # estimator to fit
                            X_train, # data to fit
                            y_train, # target variable isSepsis
                            n_jobs=-1, # use all the processors in parallel
                            verbose=1, # verbosity level
                            cv=3, # splitting strategy to compute the score N consecutive times with different splits
                            scoring="f1", # for binary targets
                            return_train_score=True)
# display the scoring
cv_lin_svm

{'fit_time': array([4.07299304, 3.80910397, 3.8720181 ]),
 'score_time': array([2.82740402, 2.72457409, 4.14820504]),
 'test_score': array([0.48504274, 0.52111226, 0.45356371]),
 'train_score': array([0.51609553, 0.50446663, 0.52691511])}

## K Nearest Neighbor Classification

In [47]:
knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(X_train, y_train)
cv_knn = cross_validate(knn,
                        X_train,
                        y_train,
                        n_jobs=-1,
                        verbose=1,
                        cv=3,
                        scoring="f1",
                        return_train_score=True)
cv_knn

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   25.9s finished


{'fit_time': array([0.01050782, 0.01596522, 0.0180161 ]),
 'score_time': array([12.07834792, 11.73511076, 11.82910824]),
 'test_score': array([0.4732334, 0.5046729, 0.430131 ]),
 'train_score': array([0.48762507, 0.45665051, 0.48996832])}

## Radius Neighbor Classifier

In [45]:
rnn = RadiusNeighborsClassifier(radius=1.0)
rnn.fit(X_train, y_train)
cv_rnn = cross_validate(rnn,
                        X_train,
                        y_train,
                        n_jobs=-1,
                        verbose=1,
                        cv=3,
                        scoring="f1",
                        return_train_score=True)
cv_rnn

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/davidmarcus/.virtualenvs/sepsisDetection /lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/davidmarcus/.virtualenvs/sepsisDetection /lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/davidmarcus/.virtualenvs/sepsisDetection /lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "/Users/davidmarcus/.virtualenvs/sepsisDetection /lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 236, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/Users/davidmarcus/.virtualenvs/sepsisDetection

{'fit_time': array([0.00887012, 0.00262618, 0.00242805]),
 'score_time': array([1.95225811, 1.41093588, 1.39185286]),
 'test_score': array([nan, nan, nan]),
 'train_score': array([0.97461929, 0.97649186, 0.97167756])}

## Decision Tree Classifier

In [51]:
dct = DecisionTreeClassifier(max_depth=None, min_samples_split=2, random_state=0)
dct.fit(X_train, y_train)
cv_dct = cross_validate(dct,
                        X_train,
                        y_train,
                        n_jobs=-1,
                        verbose=1,
                        cv=3,
                        scoring="f1",
                        return_train_score=True)
cv_dct

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.2s finished


{'fit_time': array([0.98283792, 0.91681886, 0.94060397]),
 'score_time': array([0.00798917, 0.00885034, 0.00783825]),
 'test_score': array([0.63128877, 0.64996521, 0.62068966]),
 'train_score': array([1., 1., 1.])}

## Random Forest Classifier

In [49]:
rf = RandomForestClassifier(n_estimators=10, verbose=1)
cv_rf = cross_validate(rf,
                       X_train,
                       y_train,
                       n_jobs=-1,
                       cv=3,
                       scoring="f1",
                       return_train_score=True)
cv_rf

{'fit_time': array([0.25298214, 0.24710989, 0.24381614]),
 'score_time': array([0.01477695, 0.01457   , 0.01472092]),
 'test_score': array([0.73960984, 0.74333333, 0.72379368]),
 'train_score': array([0.96296296, 0.96938776, 0.96526508])}

## Extremely Randomized Trees

In [50]:
erf = ExtraTreesClassifier(n_estimators=10, verbose=1)
cv_erf = cross_validate(erf,
                       X_train,
                       y_train,
                       n_jobs=-1,
                       cv=3,
                       scoring="f1",
                       return_train_score=True)
cv_erf


{'fit_time': array([0.176332  , 0.18742609, 0.18132997]),
 'score_time': array([0.02071595, 0.024019  , 0.02474308]),
 'test_score': array([0.64292409, 0.63450835, 0.60478469]),
 'train_score': array([1., 1., 1.])}

## Logistic Regression
- [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic%20regression#sklearn.linear_model.LogisticRegression)
- [Cross Validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html?highlight=cross_validate#sklearn-model-selection-cross-validate) to split the training set into k smaller sets

In [28]:
# set a variable to Logistic regression with verbosity
log_reg = LogisticRegression(verbose=2)
log_reg.fit(X_train, y_train)
cv_log_reg = cross_validate(log_reg,
                            X_train, # attributes
                            y_train, # labels isSepsis
                            n_jobs=-1, # use all the processors in parallel
                            verbose=1, # verbosity level
                            cv=3, # splitting strategy to compute the score N consecutive times with different splits
                            scoring="f1", # for binary targets
                            return_train_score=True) # computationally expensive, whether to include training scores on parameters impact
cv_log_reg

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.7s finished


{'fit_time': array([0.4997108 , 0.51584601, 0.5105648 ]),
 'score_time': array([0.00688839, 0.00648093, 0.01709318]),
 'test_score': array([0.50102669, 0.53532338, 0.46349206]),
 'train_score': array([0.49354672, 0.49588477, 0.50740174])}

In [37]:
xgboost = XGBClassifier(n_estimators=150, use_label_encoder=False, scale_pos_weight=12, eval_metric="aucpr", verbosity=1, disable_default_eval_metric=1)
cv_xgboost = cross_validate(xgboost,
                            X_train,
                            y_train,
                            cv=3,
                            scoring="f1",
                            return_train_score=True,
                            verbose=1)
cv_xgboost

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    5.1s finished


{'fit_time': array([2.38287592, 1.11645913, 1.46486402]),
 'score_time': array([0.01061916, 0.01619792, 0.01234484]),
 'test_score': array([0.75722543, 0.76086957, 0.74691806]),
 'train_score': array([0.96514012, 0.96811793, 0.96978022])}

In [38]:
nn = MLPClassifier(max_iter=5000, hidden_layer_sizes=(50,50,50,50), verbose=0, learning_rate="adaptive")
cv_nn = cross_validate(nn,
                       X_train,
                       y_train,
                       cv=3,
                       scoring="f1",
                       return_train_score=True,
                       verbose=1)
cv_nn

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.3min finished


{'fit_time': array([19.7811861 , 27.49344206, 32.61228228]),
 'score_time': array([0.02718282, 0.02237582, 0.02795482]),
 'test_score': array([0.62164361, 0.6309434 , 0.61001517]),
 'train_score': array([0.94042713, 0.98717035, 0.99681416])}